In [10]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from threading import Thread
import json
import platform
import re
import time
import copy

In [11]:
path = 'chromedriver.exe' if (platform.system() == 'Windows') else '/Users/jg/Desktop/develop/DataTeam/DataProcessing/product/crawling/chromedriver';
driver = webdriver.Chrome(path)

In [12]:
def writeJSON(jsonString, output_name='data.json'):
    with open(output_name,'w',encoding='UTF-8') as file:
        file.write(jsonString)

In [13]:
#categorys = [115015010000]
categorys = [115015007000, 115015008000, 115015017000, 115015015000, 115015014000, 115015009000, 115015010000, 120000000328, 120000000333, 115015013000]

In [14]:
def getUrlList():
    urlList = []
    for category in categorys:
        #카테고리별로 접속
        driver.get("http://apieu.beautynet.co.kr/goods.list.do?upperDisplayCategoryNumber=" + str(category) + "&displayCategoryNumber=0")
        
        #상품 긁기
        while True:
            html = driver.page_source
            soup = bs(html,'html.parser')
            products = driver.find_elements_by_xpath("//p[@class='pname']//a")
            for product in products:
                urlList.append(product.get_attribute("href"))
            try:
                driver.find_element_by_xpath("//a[@class='nextBtn']").click()
            except NoSuchElementException:
                break;
    return urlList
        
        #다음 페이지가 있으면 다음 페이지로 이동

In [15]:
urlList = getUrlList()


In [16]:
urlList = list(set(urlList))
print(len(urlList))

1213


In [17]:
def getItemDetailByUrl(urlList):
    result_json = []
    for idx, item in enumerate(urlList):
        driver.get(item)
        html = driver.page_source
        soup = bs(html,'html.parser')
        # dictionary 생성
        result = {'name':'', 'url':'', 'image':'', 'salePrice':'', 'originalPrice':'', 'color':'', 
                   'category':'', 'brand':'어퓨','volume':''}
        
        fullName = soup.find('h2',{'id':'goodsProdName'}).get_text().strip('[어퓨]')
        checkContainsColor = fullName.find('[')
        if checkContainsColor == -1:
            result['name'] = fullName
            result['color'] = "#"
        else:
            result['name'] = fullName[:checkContainsColor]
            result['color'] = fullName[checkContainsColor + 1: len(fullName)]
        
        result['url'] = driver.current_url
        categorys = []
        findCategory = soup.find('div',{'class':'tit_path'}).find_all('li')
        for category in findCategory:
            categorys.append(category.get_text().strip('\n'))
        result['category'] = '>'.join(categorys)
        
        #컬러가 있는지 체크
#         if soup.find('div', {'class':'colorList'}) is None:
            #단품
        images = driver.find_elements_by_xpath("//ul[@class='simg']//li//a//img")
        result['image'] = [image.get_attribute('src') for image in images]
        result['salePrice'] = soup.find('div',{'class':'itemBox priceInfo'}).find('span',{'class':'price'}).get_text()
        result['originalPrice'] = result['salePrice'] if soup.find('div',{'class':'itemBox priceInfo'}).find('span',{'class':'fixedPrice'}) is None else soup.find('div',{'class':'itemBox priceInfo'}).find('span',{'class':'fixedPrice'}).get_text()
        infoOptions = soup.find('div',{'class':'itemBox dtlInfo'}).find_all('dl')
        for option in infoOptions:
            if option.dt.get_text() == "용량":
                result['volume'] = option.dd.get_text()
                break;
        
        #display(result)
        result_json.append(result)
#         else:
#             #컬러 있음
#             #Check # of Colors
#             colorList = driver.find_elements_by_xpath("//div[@class='bx-viewport']//li")
#             for idx, color in enumerate(colorList):
#                 color.click()
#                 result_dict = copy.deepcopy(result)
#                 images = driver.find_elements_by_xpath("//ul[@class='simg']//li//a//img")
#                 result_dict['image'] = [image.get_attribute('src') for image in images]
#                 result_dict['salePrice'] = soup.find('div',{'class':'itemBox priceInfo'}).find('span',{'class':'price'}).get_text()
#                 result_dict['originalPrice'] = result_dict['salePrice'] if soup.find('div',{'class':'itemBox priceInfo'}).find('span',{'class':'fixedPrice'}) is None else soup.find('div',{'class':'itemBox priceInfo'}).find('span',{'class':'fixedPrice'}).get_text()
#                 infoOptions = soup.find('div',{'class':'itemBox dtlInfo'}).find_all('dl')
#                 for option in infoOptions:
#                     if option.dt.get_text() == "용량":
#                         result_dict['volume'] = option.dd.get_text()
#                         break;
#                 result_dict['color'] = color.find_element_by_tag_name('a').get_attribute('data-option_goods_name')
#                 result_json.append(result_dict)
#                 display(result_dict)
        if idx % (len(urlList)//50) == 0:
            print("%3.1f 퍼센트 진행중" % round(idx / len(urlList) * 100))
    return result_json

In [ ]:
result_json = getItemDetailByUrl(urlList)

0.0 퍼센트 진행중
2.0 퍼센트 진행중
4.0 퍼센트 진행중
6.0 퍼센트 진행중
8.0 퍼센트 진행중
10.0 퍼센트 진행중
12.0 퍼센트 진행중
14.0 퍼센트 진행중
16.0 퍼센트 진행중
18.0 퍼센트 진행중
20.0 퍼센트 진행중
22.0 퍼센트 진행중
24.0 퍼센트 진행중
26.0 퍼센트 진행중
28.0 퍼센트 진행중
30.0 퍼센트 진행중
32.0 퍼센트 진행중
34.0 퍼센트 진행중
36.0 퍼센트 진행중
38.0 퍼센트 진행중
40.0 퍼센트 진행중
42.0 퍼센트 진행중
44.0 퍼센트 진행중
46.0 퍼센트 진행중
47.0 퍼센트 진행중
49.0 퍼센트 진행중
51.0 퍼센트 진행중
53.0 퍼센트 진행중
55.0 퍼센트 진행중
57.0 퍼센트 진행중
59.0 퍼센트 진행중
61.0 퍼센트 진행중
63.0 퍼센트 진행중
65.0 퍼센트 진행중
67.0 퍼센트 진행중
69.0 퍼센트 진행중
71.0 퍼센트 진행중
73.0 퍼센트 진행중
75.0 퍼센트 진행중
77.0 퍼센트 진행중
79.0 퍼센트 진행중
81.0 퍼센트 진행중
83.0 퍼센트 진행중
85.0 퍼센트 진행중
87.0 퍼센트 진행중
89.0 퍼센트 진행중
91.0 퍼센트 진행중
93.0 퍼센트 진행중
95.0 퍼센트 진행중
97.0 퍼센트 진행중
99.0 퍼센트 진행중


In [123]:
output = json.dumps(result_json,ensure_ascii=False, indent='\t')

writeJSON(output, output_name = 'apieu.json')